In [ ]:
import numpy as np #numerical computation
import pandas as pd #data wrangling
import matplotlib.pyplot as plt #plotting package
#Next line helps with rendering plots
%matplotlib inline
import matplotlib as mpl #add'l plotting functionality
import seaborn as sns #a fancy plotting package
mpl.rcParams['figure.dpi'] = 400 #high res figures

Load data

In [ ]:
df = pd.read_csv('../Data/Chapter_1_cleaned_data.csv')

# Examining the Relationships between Features and the Response

In [ ]:
features_response = df.columns.tolist()

In [ ]:
features_response[:5]

In [ ]:
features_response[-5:]

In [ ]:
items_to_remove = ['ID', 'SEX', 'PAY_2', 'PAY_3', 'PAY_4', 'PAY_5', 'PAY_6',
                   'EDUCATION_CAT', 'graduate school', 'high school', 'none',
                   'others', 'university']

In [ ]:
example_list_comp = [item for item in range(5)]
example_list_comp

In [ ]:
features_response = [item for item in features_response if item not in items_to_remove]
features_response

In [ ]:
corr = df[features_response].corr()
corr.iloc[0:5,0:5]

In [ ]:
mpl.rcParams['figure.dpi'] = 400 #high res figures
sns.heatmap(corr, 
            xticklabels=corr.columns.values,
            yticklabels=corr.columns.values,
            center=0)

In [ ]:
n_points = 500
np.random.seed(seed=2)
X = np.random.uniform(low=0.0, high=10.0, size=(n_points,))
slope = 0.25
noise = 0.2
y=[]
y.append( slope * X + np.random.normal(loc=0.0, scale=noise, size=(n_points,)) )
y.append( -slope * X + np.random.normal(loc=0.0, scale=noise, size=(n_points,)) )
y.append( np.random.normal(loc=0.0, scale=noise, size=(n_points,)) )
y.append( np.sin(X/(2)*np.pi) + np.random.normal(loc=0.0, scale=noise, size=(n_points,)) )

In [ ]:
np.corrcoef(X, y[3])

In [ ]:
mpl.rcParams['figure.dpi'] = 200
fig_h, ax_h = plt.subplots(2,2)
counter = 0
correlations = []
titles = ['A', 'B', 'C', 'D']
for axis in ax_h.flatten():
    axis.scatter(X,y[counter],s=0.5)
    axis.set_xticks([])
    axis.set_yticks([])
    axis.set_title(titles[counter])
#     axis.set_title( r'$\rho = ${:0.2f}'.format(np.corrcoef(X, y[counter])[0,1]))
    correlations.append(np.corrcoef(X, y[counter])[0,1])
    counter+=1

In [ ]:
correlations

In [ ]:
# df_sample = df[['LIMIT_BAL', 'PAY_1', 'default payment next month']]\
# .sample(frac=0.1, replace=False, random_state=1, axis=0)

#Hard to see much value in either of these:
# sns.pairplot(df_sample, hue='default payment next month')
# pd.scatter_matrix(df_sample)

# Exercise 11: F-test and Univariate Feature Selection

In [ ]:
X = df[features_response].iloc[:,:-1].values
y = df[features_response].iloc[:,-1].values
print(X.shape, y.shape)

In [ ]:
from sklearn.feature_selection import f_classif

In [ ]:
[f_stat, f_p_value] = f_classif(X, y)

In [ ]:
f_test_df = pd.DataFrame({'Feature':features_response[:-1],
                          'F statistic':f_stat,
                          'p value':f_p_value})
f_test_df.sort_values('p value')

In [ ]:
from sklearn.feature_selection import SelectPercentile

In [ ]:
selector = SelectPercentile(f_classif, percentile=20)

In [ ]:
selector.fit(X, y)

In [ ]:
best_feature_ix = selector.get_support()
best_feature_ix

In [ ]:
features = features_response[:-1]

In [ ]:
best_features = [features[counter] for counter in range(len(features))
                 if best_feature_ix[counter]]

In [ ]:
best_features

# Exercise 12: Visualizing the Relationship Between Features and Response

In [ ]:
overall_default_rate = df['default payment next month'].mean()
overall_default_rate

In [ ]:
group_by_pay_mean_y = df.groupby('PAY_1').agg({'default payment next month':np.mean})
group_by_pay_mean_y

In [ ]:
axes = plt.axes()
axes.axhline(overall_default_rate, color='red')
group_by_pay_mean_y.plot(marker='x', legend=False, ax=axes)
axes.set_ylabel('Proportion of credit defaults')
axes.legend(['Entire data set', 'Groups of PAY_1'])

In [ ]:
pos_mask = y == 1
neg_mask = y == 0

In [ ]:
axes = plt.axes()
axes.hist(df.loc[neg_mask, 'LIMIT_BAL'], alpha=0.5, color='blue')
axes.hist(df.loc[pos_mask, 'LIMIT_BAL'], alpha=0.5, color='red')
axes.tick_params(axis='x', labelrotation=45)
axes.set_xlabel('Credit limit (NT$)')
axes.set_ylabel('Number of accounts')
axes.legend(['Not defaulted', 'Defaulted'])
axes.set_title('Credit limits by response variable')

In [ ]:
df['LIMIT_BAL'].max()

In [ ]:
bin_edges = list(range(0,850000,50000))
print(bin_edges[-1])

In [ ]:
mpl.rcParams['figure.dpi'] = 400 
axes = plt.axes()
axes.hist(df.loc[neg_mask, 'LIMIT_BAL'], bins=bin_edges, alpha=0.5, density=True, color='blue')
axes.hist(df.loc[pos_mask, 'LIMIT_BAL'], bins=bin_edges, alpha=0.5, density=True, color='red')
axes.tick_params(axis='x', labelrotation=45)
axes.set_xlabel('Credit limit (NT$)')
axes.set_ylabel('Proportion of accounts')
y_ticks = axes.get_yticks()
axes.set_yticklabels(np.round(y_ticks*50000,2))
axes.legend(['Not defaulted', 'Defaulted'])
axes.set_title('Normalized distributions of credit limits by response variable')

# Understanding Logistic Regression with `function` Syntax in Python and the Sigmoid Function

In [ ]:
np.mean([1, 2, 3, 4, 5])

In [ ]:
def my_mean(input_argument):
    output = sum(input_argument)/len(input_argument)
    return(output)

In [ ]:
my_mean([1, 2, 3, 4, 5])

In [ ]:
my_mean(input_argument=[1, 2, 3])

In [ ]:
np.exp(1)

In [ ]:
np.exp(0)

In [ ]:
X_exp = np.linspace(-4,4,81)
print(X_exp[:5])
print(X_exp[-5:])

In [ ]:
Y_exp = np.exp(X_exp)
Y_exp[:5]

# Exercise 13: Plotting the Sigmoid Function

In [ ]:
plt.plot(X_exp, Y_exp)
plt.title('Plot of $e^X$')

In [ ]:
Y_exp = np.exp(-X_exp)
plt.plot(X_exp, Y_exp)
plt.title('Plot of $e^{-X}$')

In [ ]:
def sigmoid(X):
    Y = 1 / (1 + np.exp(-X))
    return Y

In [ ]:
X_sig = np.linspace(-7,7,141)
Y_sig = sigmoid(X_sig)
plt.plot(X_sig,Y_sig)
plt.yticks(np.linspace(0,1,11))
plt.grid()
plt.title('The sigmoid function')

# Scope of Functions

In [ ]:
Y

In [ ]:
example_global_variable = 1

In [ ]:
def example_function():
    output = example_global_variable + 1
    return(output)

In [ ]:
example_function()

# Exercise 14: Examining the Appropriateness of Features for Logistic Regression

In [ ]:
group_by_pay_mean_y

In [ ]:
p = group_by_pay_mean_y['default payment next month'].values
q = 1-p
print(p)
print(q)

In [ ]:
odds_ratio = p/q
odds_ratio

In [ ]:
log_odds = np.log(odds_ratio)
log_odds

In [ ]:
group_by_pay_mean_y.index

In [ ]:
plt.plot(group_by_pay_mean_y.index, log_odds, '-x')
plt.ylabel('Log odds of default')
plt.xlabel('Values of PAY_1')

# From Logistic Regression Coefficients to Predictions Using the Sigmoid

In [ ]:
X_sig = np.linspace(-7,7,141)
Y_sig = sigmoid(X_sig)
plt.plot(X_sig,Y_sig)
plt.yticks(np.linspace(0,1,11))
plt.grid()
plt.title('The sigmoid function')
plt.xlabel('$X = \Theta_0 + \Theta_1 X_1 + \Theta_2 X_2$')
plt.ylabel('$p$')

# Exercise 15: Linear Decision Boundary of Logistic Regression

In [ ]:
np.random.seed(seed=6)
X_1_pos = np.random.uniform(low=1, high=7, size=(20,1))
print(X_1_pos[0:3])
X_1_neg = np.random.uniform(low=3, high=10, size=(20,1))
print(X_1_neg[0:3])
X_2_pos = np.random.uniform(low=1, high=7, size=(20,1))
print(X_1_pos[0:3])
X_2_neg = np.random.uniform(low=3, high=10, size=(20,1))
print(X_1_neg[0:3])

In [ ]:
plt.scatter(X_1_pos, X_2_pos, color='red', marker='x')
plt.scatter(X_1_neg, X_2_neg, color='blue', marker='x')
plt.xlabel('$X_1$')
plt.ylabel('$X_2$')
plt.legend(['Positive class', 'Negative class'])

In [ ]:
X = np.block([[X_1_pos, X_2_pos], [X_1_neg, X_2_neg]])

In [ ]:
print(X.shape)
print(X[0:3])

In [ ]:
y = np.vstack((np.ones((20,1)), np.zeros((20,1)))).reshape(40,)
print(y[0:5])
print(y[-5:])

In [ ]:
from sklearn.linear_model import LogisticRegression

In [ ]:
example_lr = LogisticRegression(solver='liblinear')

In [ ]:
example_lr

In [ ]:
example_lr.fit(X, y)

In [ ]:
y_pred = example_lr.predict(X)

In [ ]:
positive_indices = [counter for counter in range(len(y_pred)) if y_pred[counter]==1]
negative_indices = [counter for counter in range(len(y_pred)) if y_pred[counter]==0]

In [ ]:
positive_indices

In [ ]:
plt.scatter(X_1_pos, X_2_pos, color='red', marker='x')
plt.scatter(X_1_neg, X_2_neg, color='blue', marker='x')
plt.scatter(X[positive_indices,0], X[positive_indices,1], s=150, marker='o',
            edgecolors='red', facecolors='none')
plt.scatter(X[negative_indices,0], X[negative_indices,1], s=150, marker='o',
            edgecolors='blue', facecolors='none')
plt.xlabel('$X_1$')
plt.ylabel('$X_2$')
plt.legend(['Positive class', 'Negative class', 'Positive predictions', 'Negative predictions'])

$\theta_0 + \theta_1 X_1 + \theta_2 X_2 = 0$ is the equation for the decision boundary line, for a threshold of 0.5. Solving for "$y = mx + b$" form,

$X_2 = -\frac{\theta_1}{\theta_2}X_1 - \frac{\theta_0}{\theta_2}$

In [ ]:
theta_1 = example_lr.coef_[0][0]
theta_2 = example_lr.coef_[0][1]
print(theta_1, theta_2)

In [ ]:
example_lr.intercept_

In [ ]:
theta_0 = example_lr.intercept_

In [ ]:
X_1_decision_boundary = np.array([0, 10])

In [ ]:
X_2_decision_boundary = -(theta_1/theta_2)*X_1_decision_boundary - (theta_0/theta_2)

In [ ]:
pos_true = plt.scatter(X_1_pos, X_2_pos, color='red', marker='x', label='Positive class')
neg_true = plt.scatter(X_1_neg, X_2_neg, color='blue', marker='x', label='Negative class')
pos_pred = plt.scatter(X[positive_indices,0], X[positive_indices,1], s=150, marker='o',
            edgecolors='red', facecolors='none', label='Positive predictions')
neg_pred = plt.scatter(X[negative_indices,0], X[negative_indices,1], s=150, marker='o',
            edgecolors='blue', facecolors='none', label='Negative predictions')
dec = plt.plot(X_1_decision_boundary, X_2_decision_boundary, 'k-', label='Decision boundary')
plt.xlabel('$X_1$')
plt.ylabel('$X_2$')
plt.legend(loc=[0.25, 1.05])

# Activity 3: Fitting a Logistic Regression Model and Directly Using the Coefficients

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
df[['PAY_1', 'LIMIT_BAL']].values, df['default payment next month'].values,
test_size=0.2, random_state=24)

In [ ]:
print(X_train.shape)
print(X_test.shape)

In [ ]:
from sklearn.linear_model import LogisticRegression

In [ ]:
lr_model = LogisticRegression(solver='liblinear')

In [ ]:
lr_model

In [ ]:
lr_model.fit(X_train, y_train)

In [ ]:
y_pred = lr_model.predict(X_test)

In [ ]:
y_pred_proba = lr_model.predict_proba(X_test)

In [ ]:
print(lr_model.coef_, lr_model.intercept_)

In [ ]:
np.ones((X_test.shape[0],1)).shape

In [ ]:
ones_and_features = np.hstack([np.ones((X_test.shape[0],1)), X_test])
ones_and_features

In [ ]:
intercept_and_coefs = np.concatenate([lr_model.intercept_.reshape(1,1), lr_model.coef_], axis=1)
intercept_and_coefs

In [ ]:
X_lin_comb = np.dot(intercept_and_coefs, np.transpose(ones_and_features))

In [ ]:
y_pred_proba_manual = sigmoid(X_lin_comb)

In [ ]:
y_pred_manual = y_pred_proba_manual >= 0.5

In [ ]:
y_pred.shape

In [ ]:
y_pred_manual.shape

In [ ]:
np.array_equal(y_pred.reshape(1,-1), y_pred_manual)

In [ ]:
from sklearn.metrics import roc_auc_score

In [ ]:
y_test.shape

In [ ]:
y_pred_proba_manual.shape

In [ ]:
roc_auc_score(y_test, y_pred_proba_manual.reshape(y_pred_proba_manual.shape[1],))

In [ ]:
roc_auc_score(y_test, y_pred_proba[:,1])